# 1. Calculation of indicator record value: 

This notebook contains a QA of the queries implemented for the impact calculation in the main LG application. The main purpose is to update these calculations according to the improvements on the methodology: https://docs.google.com/document/d/1IDuYWOllQ2fTf2ZeBUmOtEZpqht2rqMot3G9W3CjEjE/edit#

As part of new strategy the indicator record entity will include:

    - Indicator record value: value of impact in my geometry
    - Indicator recod scaler: equivalent to total commodity production in my location
    - Pointer: h3 table and column name to distribute the impact

In this notebook we will also be covering two approaches to compute the indicator record value:

    a. Get the total indicator record value in my geometry by summing the impact in all the hexagons within my geometry.
    b. Get the indicator record value in my geometry by computing the average risk in all the hexagons within my geometry and multiply that value by the total volume.

In [7]:
# import libraries

import pandas as pd
from psycopg2.pool import ThreadedConnectionPool

In [8]:
# set env
## env file for gcs upload
env_path = ".env"
with open(env_path) as f:
    env = {}
    for line in f:
        env_key, _val = line.split("=", 1)
        env_value = _val.split("\n")[0]
        env[env_key] = env_value

# list(env.keys())

# set conexion to local ddbb
postgres_thread_pool = ThreadedConnectionPool(
    1,
    50,
    host=env["API_POSTGRES_HOST"],
    port=env["API_POSTGRES_PORT"],
    user=env["API_POSTGRES_USERNAME"],
    password=env["API_POSTGRES_PASSWORD"],
)

# get list of sourcing records to iterate:
conn = postgres_thread_pool.getconn()
cursor = conn.cursor()

##  Compute indicator record value as sum of impacts in area:

As summary the formulas to compute each of the landgriffon impact indicators are shown below:
    
### Water impacts 

        water imapct (m3/yr) = Sum(BWF (mm/yr * tonne) * 0.001 (unit conversion)) * Volume (tonnes)   
                                             /                                       /
                                      per cell                                per region
### Land impact:

        Land impact (ha/yr)  = (Harvested area (ha) / Production (tonnes)) *  Volume (tonnes)
     
    
### Deforestation:

        Deforestation impact (ha/yr) = land impact (ha/yr) * deforestation mask (unitless)

### Carbon:

        Carbon impact (tCO2e/yr) = net forest carbo emisions (tCO2e/ha) * Deforestation impact (ha/yr)
         
    
### Biodiversity:

        Biodiversity  impact  (PDF/yr)  = PSL(PDF m⁻²) * 10⁴(m² ha⁻¹) * Deforestation impact (ha/yr)

         

In [9]:
def psql(query):
    try:
        cursor.execute(query)
        return cursor.fetchall()
    except Exception as e:
        conn.rollback()
        print(e)

In [12]:
SQL_GET_H3_UNCOMPACT_GEO_REGION = """
CREATE OR REPLACE FUNCTION get_h3_uncompact_geo_region(geo_region_id uuid, h3_resolution int)
RETURNS TABLE (h3index h3index) AS 
$$
    SELECT h3_uncompact(geo_region."h3Compact"::h3index[], h3_resolution) h3index
    FROM geo_region WHERE geo_region.id = geo_region_id
$$ 
LANGUAGE SQL;
"""

SQL_SUM_H3_GRID_OVER_GEO_REGION = (
    SQL_GET_H3_UNCOMPACT_GEO_REGION
    + """
CREATE OR REPLACE FUNCTION sum_h3_grid_over_georegion(
    geo_region_id uuid, 
    h3_resolution int,
    h3_table_name varchar, 
    h3_column_name varchar
)
RETURNS float AS
$$
    DECLARE
        sum float;
    BEGIN
        EXECUTE format(
            'SELECT sum(h3grid.%I)
                FROM
                    get_h3_uncompact_geo_region($1, $2) geo_region
                    INNER JOIN %I h3grid ON h3grid.h3index = geo_region.h3index;
            ', h3_column_name, h3_table_name)
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;
"""
)

SQL_SUMPROD_H3_GRIDS_OVER_GEOREGION = (
    SQL_GET_H3_UNCOMPACT_GEO_REGION
    + """
CREATE OR REPLACE FUNCTION sumprod_h3_grids_over_georegion(
    geo_region_id uuid,
    h3_resolution int,
    h3_table_name_1 varchar,
    h3_column_name_1 varchar,
    h3_table_name_2 varchar,
    h3_column_name_2 varchar
)
RETURNS float AS
$$
    DECLARE
        sumprod float;
    BEGIN
        EXECUTE format(
            'SELECT sum(h3grid_1.%I * h3grid_2.%I)
                FROM
                    get_h3_uncompact_geo_region($1, $2) geo_region
                    INNER JOIN %I h3grid_1 ON h3grid_1.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_2 ON h3grid_2.h3index = geo_region.h3index;
            ', h3_column_name_1, h3_column_name_2, h3_table_name_1, h3_table_name_2)
            USING geo_region_id, h3_resolution
            INTO sumprod;
        RETURN sumprod;
    END;
$$
LANGUAGE plpgsql;
"""
)

SQL_GET_H3_TABLE_COLUMN_FOR_MATERIAL = """
CREATE OR REPLACE FUNCTION get_h3_table_column_for_material(material_id uuid, h3_data_type material_to_h3_type_enum)
RETURNS TABLE (h3_table_name varchar, h3_column_name varchar, h3_resolution int) AS
$$
    SELECT h3_data."h3tableName", h3_data."h3columnName", h3_data."h3resolution"
    FROM h3_data
        INNER JOIN material_to_h3 ON material_to_h3."h3DataId" = h3_data.id
    WHERE material_to_h3."materialId" = material_id 
        AND material_to_h3.type = h3_data_type
    LIMIT 1;
$$
LANGUAGE SQL;
"""

SQL_SUM_MATERIAL_OVER_GEO_REGION = (
    SQL_SUM_H3_GRID_OVER_GEO_REGION
    + SQL_GET_H3_TABLE_COLUMN_FOR_MATERIAL
    + """
CREATE OR REPLACE FUNCTION sum_material_over_georegion(
    geo_region_id uuid, 
    material_id uuid,
    h3_data_type material_to_h3_type_enum
)
RETURNS float AS
$$
    DECLARE
        h3_table_name varchar;
        h3_column_name varchar;
        h3_resolution integer;
        sum float;

    BEGIN
        -- Get h3data table name and column name for given material
        SELECT * INTO h3_table_name, h3_column_name, h3_resolution
        FROM get_h3_table_column_for_material(material_id, h3_data_type);

        -- Sum table column over region
        SELECT sum_h3_grid_over_georegion(geo_region_id, h3_resolution, h3_table_name, h3_column_name) 
        INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;
"""
)

SQL_SUM_WEIGHTED_DEFORESTATION_OVER_GEO_REGION = """
CREATE OR REPLACE FUNCTION sum_weighted_deforestation_over_georegion(
    geo_region_id uuid, 
    material_id uuid,
    h3_data_type material_to_h3_type_enum
)
RETURNS float AS
$$
    DECLARE
        material_h3_table_name varchar;
        material_h3_column_name varchar;
        h3_resolution integer;
        deforestation_h3_table_name varchar := 'h3_grid_deforestation_global';
        deforestation_h3_column_name varchar := 'hansenLoss2019';
        sum float;

    BEGIN
        -- Get h3data table name and column name for given material
        SELECT * INTO material_h3_table_name, material_h3_column_name, h3_resolution
        FROM get_h3_table_column_for_material(material_id, h3_data_type);

        -- Sum table column over region
        EXECUTE format(
            'SELECT sum(h3grid_mat.%I * h3grid_def.%I)
                FROM
                    get_h3_uncompact_geo_region($1, $2) geo_region
                    INNER JOIN %I h3grid_mat ON h3grid_mat.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_def ON h3grid_def.h3index = geo_region.h3index;
            ', material_h3_column_name, deforestation_h3_column_name, material_h3_table_name, deforestation_h3_table_name)
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;
"""

SQL_SUM_WEIGHTED_BIODIVERSITY_OVER_GEO_REGION = """
CREATE OR REPLACE FUNCTION sum_weighted_biodiversity_over_georegion(
    geo_region_id uuid, 
    material_id uuid,
    h3_data_type material_to_h3_type_enum
)
RETURNS float AS
$$
    DECLARE
        material_h3_table_name varchar;
        material_h3_column_name varchar;
        h3_resolution integer;
        deforestation_h3_table_name varchar := 'h3_grid_deforestation_global';
        deforestation_h3_column_name varchar := 'hansenLoss2019';
        bio_h3_table_name varchar := 'h3_grid_bio_global';
        bio_h3_column_name varchar := 'lciaPslRPermanentCrops';
        sum float;

    BEGIN
        -- Get h3data table name and column name for given material --
        SELECT * INTO material_h3_table_name, material_h3_column_name, h3_resolution
        FROM get_h3_table_column_for_material(material_id, h3_data_type);

        -- Sum deforestation times biodiversity where material is produced --
        EXECUTE format(
                'SELECT sum(h3grid_mat.%I * h3grid_def.%I * h3grid_bio.%I * (1/0.0001))
                    FROM get_h3_uncompact_geo_region($1, $2) geo_region
                        INNER JOIN %I h3grid_mat ON h3grid_mat.h3index = geo_region.h3index
                        INNER JOIN %I h3grid_def ON h3grid_def.h3index = geo_region.h3index
                        INNER JOIN %I h3grid_bio ON h3grid_bio.h3index = geo_region.h3index;', 
                material_h3_column_name, 
                deforestation_h3_column_name, 
                bio_h3_column_name,
                material_h3_table_name,
                deforestation_h3_table_name,
                bio_h3_table_name
            )
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;
"""

SQL_SUM_WEIGHTED_CARBON_OVER_GEO_REGION = """
CREATE OR REPLACE FUNCTION sum_weighted_carbon_over_georegion(
    geo_region_id uuid, 
    material_id uuid,
    h3_data_type material_to_h3_type_enum
)
RETURNS float AS
$$
    DECLARE
        material_h3_table_name varchar;
        material_h3_column_name varchar;
        h3_resolution integer;
        deforestation_h3_table_name varchar := 'h3_grid_deforestation_global';
        deforestation_h3_column_name varchar := 'hansenLoss2019';
        carbon_h3_table_name varchar := 'h3_grid_carbon_global';
        carbon_h3_column_name varchar := 'earthstat2000GlobalHectareEmissions';
        sum float;

    BEGIN
        -- Get h3data table name and column name for given material --
        SELECT * INTO material_h3_table_name, material_h3_column_name, h3_resolution
        FROM get_h3_table_column_for_material(material_id, h3_data_type);

        -- Sum deforestation times carbon where material is produced --
        EXECUTE format(
                'SELECT sum(h3grid_mat.%I * h3grid_def.%I * h3grid_carbon.%I)
                    FROM get_h3_uncompact_geo_region($1, $2) geo_region
                        INNER JOIN %I h3grid_mat ON h3grid_mat.h3index = geo_region.h3index
                        INNER JOIN %I h3grid_def ON h3grid_def.h3index = geo_region.h3index
                        INNER JOIN %I h3grid_carbon ON h3grid_carbon.h3index = geo_region.h3index;', 
                material_h3_column_name, 
                deforestation_h3_column_name, 
                carbon_h3_column_name,
                material_h3_table_name,
                deforestation_h3_table_name,
                carbon_h3_table_name
            )
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;
"""

SQL_SUM_WEIGHTED_WATER_OVER_GEO_REGION = """
CREATE OR REPLACE FUNCTION sum_weighted_water_over_georegion(
    geo_region_id uuid
)
RETURNS float AS
$$
    DECLARE
        water_h3_table_name varchar := 'h3_grid_wf_global';
        water_h3_column_name varchar := 'wfBltotMmyrT';
        h3_resolution integer := 6;
        sum float;

    BEGIN

        -- Sum deforestation times carbon where material is produced --
        EXECUTE format(
                'SELECT sum(h3grid_water.%I * 0.001)
                    FROM get_h3_uncompact_geo_region($1, $2) geo_region
                        INNER JOIN %I h3grid_water ON h3grid_water.h3index = geo_region.h3index;', 
                water_h3_column_name, 
                water_h3_table_name
            )
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;
"""

In [13]:
%%time
sourcing_locations = pd.read_sql_query(
    SQL_SUM_MATERIAL_OVER_GEO_REGION
    + SQL_SUM_WEIGHTED_DEFORESTATION_OVER_GEO_REGION
    + SQL_SUM_WEIGHTED_BIODIVERSITY_OVER_GEO_REGION
    + SQL_SUM_WEIGHTED_CARBON_OVER_GEO_REGION
    + SQL_SUM_WEIGHTED_WATER_OVER_GEO_REGION
    + """
    SELECT
        id,
        sum_material_over_georegion("geoRegionId", "materialId", 'producer') as production,
        sum_material_over_georegion("geoRegionId", "materialId", 'harvest') as harvested_area,
        sum_weighted_deforestation_over_georegion("geoRegionId", "materialId", 'harvest') as raw_deforestation,
        sum_weighted_biodiversity_over_georegion("geoRegionId", "materialId", 'harvest') as raw_biodiversity,
        sum_weighted_carbon_over_georegion("geoRegionId", "materialId", 'harvest') as raw_carbon,
        sum_weighted_water_over_georegion("geoRegionId") as raw_water
    FROM
        sourcing_location
    """,
    conn,
)

sourcing_locations

CPU times: user 8.05 ms, sys: 1.5 ms, total: 9.55 ms
Wall time: 41.8 s


,id,production,harvested_area,raw_deforestation,raw_biodiversity,raw_carbon,raw_water
0,8dce097b-0376-4cc8-b662-793cbe5b1383,3.197394e+04,3.149400e+04,2.965525e+04,4.138369e+09,6.836626e+04,1.231743e+08
1,90f9d8d6-927e-4f76-99d0-29b0d281001b,9.303148e+04,1.368110e+05,1.362851e+05,1.145257e+10,9.311942e+05,6.977503e+06
2,337f942e-155d-477c-b35e-247b84144491,9.806474e+04,1.872913e+04,1.861770e+04,1.744553e+09,1.887308e+05,4.829850e+09
3,68211ccb-625d-461e-821e-ba5923d7c51f,5.259789e+05,7.618299e+05,5.864385e+05,6.418257e+10,4.192368e+06,2.486174e+11
4,49e12f5e-b9eb-4b8a-be62-fa548fc1352f,4.661946e+06,2.990404e+06,2.820604e+06,1.862220e+11,9.925858e+06,1.590841e+10
5,caf46250-e864-4ab9-a6c6-fad6fd7d0163,1.928499e+06,2.834290e+06,2.764138e+06,1.986828e+11,2.145601e+07,4.829850e+09
6,0d237cb4-d5df-402a-a73b-b72fb12c73fe,3.775885e+06,5.318659e+06,5.066399e+06,2.974967e+11,6.634060e+07,8.478840e+12
7,baa2180c-74bd-4764-a049-2469c6047c01,2.517753e+04,1.916179e+04,1.906270e+04,8.411893e+08,1.010275e+05,2.486174e+11
8,f1ff3505-4f05-4b5d-bb76-2ddd8115abbf,3.699012e+04,5.487284e+05,5.310155e+05,4.079312e+10,1.852687e+06,1.590841e+10
9,2264a013-60b7-4988-b39c-41dcf0282979,1.710592e+02,1.168384e+06,9.546428e+05,9.483591e+10,7.503546e+06,2.486174e+11


In [14]:
sourcing_records = pd.read_sql_query(
    SQL_SUM_MATERIAL_OVER_GEO_REGION
    + SQL_SUM_WEIGHTED_DEFORESTATION_OVER_GEO_REGION
    + SQL_SUM_WEIGHTED_BIODIVERSITY_OVER_GEO_REGION
    + SQL_SUM_WEIGHTED_CARBON_OVER_GEO_REGION
    + SQL_SUM_WEIGHTED_WATER_OVER_GEO_REGION
    + """
    SELECT
        sr.id,
        sr.tonnage,
        sr.year,
        sl.id,
        sl.production,
        sl.harvested_area,
        sl.raw_deforestation,
        sl.raw_biodiversity,
        sl.raw_carbon,
        sl.raw_water
    FROM
        sourcing_records sr
        INNER JOIN
            (
                SELECT
                    id,
                    sum_material_over_georegion("geoRegionId", "materialId", 'producer') as production,
                    sum_material_over_georegion("geoRegionId", "materialId", 'harvest') as harvested_area,
                    sum_weighted_deforestation_over_georegion("geoRegionId", "materialId", 'harvest') as raw_deforestation,
                    sum_weighted_biodiversity_over_georegion("geoRegionId", "materialId", 'harvest') as raw_biodiversity,
                    sum_weighted_carbon_over_georegion("geoRegionId", "materialId", 'harvest') as raw_carbon,
                    sum_weighted_water_over_georegion("geoRegionId") as raw_water
                FROM
                    sourcing_location
            ) as sl
            on sr."sourcingLocationId" = sl.id
""",
    conn,
)

sourcing_records["land_per_ton"] = (
    sourcing_records["harvested_area"] / sourcing_records["production"]
)

sourcing_records["deforestation_per_ha_landuse"] = (
    sourcing_records["raw_deforestation"] / sourcing_records["harvested_area"]
)
sourcing_records["bio_per_ha_landuse"] = (
    sourcing_records["raw_biodiversity"] / sourcing_records["harvested_area"]
)
sourcing_records["carbon_per_ha_landuse"] = (
    sourcing_records["raw_carbon"] / sourcing_records["harvested_area"]
)
sourcing_records["land_use"] = sourcing_records["land_per_ton"] * sourcing_records["tonnage"]

sourcing_records["deforestation"] = (
    sourcing_records["deforestation_per_ha_landuse"] * sourcing_records["land_use"]
)
sourcing_records["biodiversity_loss"] = (
    sourcing_records["bio_per_ha_landuse"] * sourcing_records["land_use"]
)
sourcing_records["carbon_loss"] = (
    sourcing_records["carbon_per_ha_landuse"] * sourcing_records["land_use"]
)
sourcing_records["water_impact"] = sourcing_records["raw_water"] * sourcing_records["tonnage"]

# Farm impact scaler = production
# Land use change impact scaler = harvested_area

sourcing_records.to_csv("test_impact_calc.csv")

sourcing_records

,id,tonnage,year,id,production,harvested_area,raw_deforestation,raw_biodiversity,raw_carbon,raw_water,land_per_ton,deforestation_per_ha_landuse,bio_per_ha_landuse,carbon_per_ha_landuse,land_use,deforestation,biodiversity_loss,carbon_loss,water_impact
0,4057312f-131b-44a3-86cb-4ff6253026ce,650.0,2010,8dce097b-0376-4cc8-b662-793cbe5b1383,31973.941406,31494.000000,29655.245346,4.138369e+09,68366.264001,1.231743e+08,0.984990,0.941616,131401.821233,2.170771,640.243245,602.863101,8.412913e+07,1389.821512,8.006332e+10
1,2280336e-f744-4ab0-a1e6-9c3a9e83c62b,657.0,2011,8dce097b-0376-4cc8-b662-793cbe5b1383,31973.941406,31494.000000,29655.245346,4.138369e+09,68366.264001,1.231743e+08,0.984990,0.941616,131401.821233,2.170771,647.138172,609.355473,8.503513e+07,1404.788821,8.092554e+10
2,1926c1dc-caf5-467b-93da-b958ca512865,664.0,2012,8dce097b-0376-4cc8-b662-793cbe5b1383,31973.941406,31494.000000,29655.245346,4.138369e+09,68366.264001,1.231743e+08,0.984990,0.941616,131401.821233,2.170771,654.033099,615.847845,8.594114e+07,1419.756130,8.178776e+10
3,c8266f82-5b40-4ffb-a6c8-42cf445064f4,671.0,2013,8dce097b-0376-4cc8-b662-793cbe5b1383,31973.941406,31494.000000,29655.245346,4.138369e+09,68366.264001,1.231743e+08,0.984990,0.941616,131401.821233,2.170771,660.928027,622.340217,8.684715e+07,1434.723438,8.264998e+10
4,b6488db3-165a-4123-9313-abc91da76381,678.0,2014,8dce097b-0376-4cc8-b662-793cbe5b1383,31973.941406,31494.000000,29655.245346,4.138369e+09,68366.264001,1.231743e+08,0.984990,0.941616,131401.821233,2.170771,667.822954,628.832589,8.775315e+07,1449.690747,8.351221e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,59758b83-4a05-4a94-a6ca-eb3a7c4ca327,422.0,2016,094a857f-bb63-4081-a953-fa8c769aff50,540.700012,136.100006,0.000000,0.000000e+00,0.000000,2.078879e+06,0.251711,0.000000,0.000000,0.000000,106.221937,0.000000,0.000000e+00,0.000000,8.772871e+08
491,01dec32c-5dd5-46c5-912c-c48bcdad9ebb,426.0,2017,094a857f-bb63-4081-a953-fa8c769aff50,540.700012,136.100006,0.000000,0.000000e+00,0.000000,2.078879e+06,0.251711,0.000000,0.000000,0.000000,107.228780,0.000000,0.000000e+00,0.000000,8.856026e+08
492,2eddcca3-a43f-4b65-895f-16bc6500b78a,430.0,2018,094a857f-bb63-4081-a953-fa8c769aff50,540.700012,136.100006,0.000000,0.000000e+00,0.000000,2.078879e+06,0.251711,0.000000,0.000000,0.000000,108.235623,0.000000,0.000000e+00,0.000000,8.939181e+08
493,b069f4f8-9ea7-49fc-a9e7-d911459b7d2f,434.0,2019,094a857f-bb63-4081-a953-fa8c769aff50,540.700012,136.100006,0.000000,0.000000e+00,0.000000,2.078879e+06,0.251711,0.000000,0.000000,0.000000,109.242466,0.000000,0.000000e+00,0.000000,9.022336e+08


In [7]:
query1 = (
    SQL_SUM_MATERIAL_OVER_GEO_REGION
    + SQL_SUM_WEIGHTED_DEFORESTATION_OVER_GEO_REGION
    + SQL_SUM_WEIGHTED_BIODIVERSITY_OVER_GEO_REGION
    + """
EXPLAIN ANALYZE 
SELECT 
    sum_material_over_georegion('68ed9c70-0f01-495f-9a53-68e5cb35c7ca', '0d7b1be5-dc86-47b8-ba3a-25190a275011', 'producer'),
    sum_weighted_deforestation_over_georegion('68ed9c70-0f01-495f-9a53-68e5cb35c7ca', '0d7b1be5-dc86-47b8-ba3a-25190a275011', 'harvest'),
    sum_weighted_bio_over_georegion('68ed9c70-0f01-495f-9a53-68e5cb35c7ca', '0d7b1be5-dc86-47b8-ba3a-25190a275011', 'harvest')
"""
)
print(psql(query1))

"""
EXPLAIN ANALYZE
SELECT sum(prod."earthstat2000GlobalRubberProduction") as value
FROM
    (SELECT h3_uncompact(geo_region."h3Compact"::h3index[], 6) h3index
        FROM geo_region WHERE geo_region.id = '68ed9c70-0f01-495f-9a53-68e5cb35c7ca'
    ) geom
    INNER JOIN h3_grid_earthstat2000_global_prod prod
    on geom.h3index = prod.h3index
"""
# print(psql(query2))

%timeit psql("SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-68e5cb35c7ca', '0d7b1be5-dc86-47b8-ba3a-25190a275011', 'producer')")
%timeit psql("""SELECT sum(prod."earthstat2000GlobalRubberProduction") as value FROM (SELECT h3_uncompact(geo_region."h3Compact"::h3index[], 6) h3index FROM geo_region WHERE geo_region.id = '68ed9c70-0f01-495f-9a53-68e5cb35c7ca') geom INNER JOIN h3_grid_earthstat2000_global_prod prod on geom.h3index = prod.h3index""")

function sum_weighted_bio_over_georegion(unknown, unknown, unknown) does not exist
LINE 154:     sum_weighted_bio_over_georegion('68ed9c70-0f01-495f-9a53...
              ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

None
function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^


function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No 

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No 

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No 

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No 

...

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No 

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No 

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No 

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

function sum_material_over_georegion(unknown, unknown, unknown) does not exist
LINE 1: SELECT sum_material_over_georegion('68ed9c70-0f01-495f-9a53-...
               ^
HINT:  No 

244 µs ± 4.43 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
